In [1]:
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/path/to/ffmpeg"

from fer import FER
from fer import Video
import pandas as pd
import matplotlib.pyplot as plt
video_file: str = "JPowell.mp4"

face_detector: FER = FER(mtcnn=True)

# preprocess video file
processed_video = Video(video_file=video_file)

processing_data = processed_video.analyze(face_detector,display=True)

df= pd.DataFrame(processing_data)

INFO:fer:29.97 fps, 1169 frames, 39.01 seconds
INFO:fer:Making directories at output
INFO:fer:Deleted pre-existing output/JPowell_output.mp4
 20%|██        | 239/1169 [00:46<03:01,  5.12frames/s]
INFO:fer:Completed analysis: saved to output/JPowell_output.mp4
INFO:fer:Starting to Zip
INFO:fer:Compressing: 20%
INFO:fer:Compressing: 41%
INFO:fer:Compressing: 62%
INFO:fer:Compressing: 83%
INFO:fer:Zip has finished


In [2]:
df= pd.DataFrame(processing_data)

In [3]:
# use the data  to decide the our execution/approach
import numpy as np
df.dropna(axis=1,inplace=True)
df['minutes']=(np.arange(len(df))//60)+1
df_grouped_by_minute = df.groupby('minutes').sum()
df_grouped_by_minute.head()


,box0,angry0,disgust0,fear0,happy0,sad0,surprise0,neutral0
minutes,,,,,,,,
1,"[539, 120, 170, 214, 540, 119, 168, 212, 536, ...",20.41,0.00,4.25,2.21,3.62,0.43,29.05
2,"[514, 135, 169, 218, 514, 134, 169, 218, 515, ...",22.38,0.00,2.06,1.90,3.75,0.20,29.56
3,"[526, 134, 173, 217, 532, 138, 163, 213, 534, ...",23.58,0.06,2.25,3.87,3.63,0.79,25.76
4,"[548, 130, 171, 221, 549, 132, 169, 218, 549, ...",16.19,0.00,0.94,2.85,3.30,0.37,32.32


In [4]:
# df['positive']=df['happy0']+df['surprise0']
# df['negative']=df['angry0']+df['disgust0']+df['fear0']+df['sad0']

# # Correct the dictionary for renaming
# neutral_dict = {'neutral0': 'neutral'}

# # Use the rename method with the columns parameter
# df.rename(columns=neutral_dict, inplace=True)
df_grouped_by_minute=df_grouped_by_minute.reset_index(drop=True)

In [5]:
# df[(df['negative']>=0.4) & (df['negative']>df['neutral'])].reset_index(drop=True)

In [6]:
import requests
import pandas as pd
API_KEY = 'JDRUHFYE77G75DS7'
symbol = 'SPY'
interval = '1min'

# url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={API_KEY}'
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&extended_hours=false&month=2024-01&outputsize=full&apikey={API_KEY}"
r = requests.get(url)
data = r.json()

time_series_key = next(key for key in data.keys() if "Time Series" in key)  # Find the correct key for the time series data

finance_data = pd.DataFrame(data[time_series_key]).T

finance_data.index = pd.to_datetime(finance_data.index)
for column in finance_data.columns:
    finance_data[column] = pd.to_numeric(finance_data[column], errors='coerce')

column_mapping = {
    '1. open': 'Open',
    '2. high': 'High',
    '3. low': 'Low',
    '4. close': 'Close',
    '5. volume': 'Volume'
}
finance_data.rename(columns=column_mapping, inplace=True)
finance_data.sort_index(inplace=True)

In [7]:
finance_data=finance_data.iloc[:19,:]
finance_data

,Open,High,Low,Close,Volume
2024-01-02 09:30:00,472.160,472.670,472.050,472.530,1205526
2024-01-02 09:31:00,472.530,472.700,472.365,472.681,756655
2024-01-02 09:32:00,472.680,472.800,472.650,472.780,693889
2024-01-02 09:33:00,472.760,472.770,472.480,472.690,404264
2024-01-02 09:34:00,472.685,472.780,472.550,472.670,250698
2024-01-02 09:35:00,472.670,472.690,472.480,472.650,267034
2024-01-02 09:36:00,472.640,472.740,472.530,472.635,317003
2024-01-02 09:37:00,472.620,472.700,472.450,472.470,209838
2024-01-02 09:38:00,472.460,472.480,472.010,472.080,553233
2024-01-02 09:39:00,472.090,472.130,471.880,471.920,355159


In [8]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets,linear_model,metrics
from sklearn.linear_model import LinearRegression



# Now, let's perform linear regression with the dataframe's features.
X = df_grouped_by_minute.drop(['box0'], axis=1)
y = finance_data['Close'].reset_index(drop=True)

# Create the model
model = LinearRegression()

# Fit the model
model.fit(X, y)

# Get the model's coefficients and intercept
coefficients = model.coef_
intercept = model.intercept_

# Prepare a DataFrame for the coefficients
coeff_df = pd.DataFrame(coefficients, X.columns, columns=['Coefficient'])

coeff_df, intercept


ValueError: Found input variables with inconsistent numbers of samples: [4, 19]

: 

In [ ]:
from sklearn.metrics import r2_score
from scipy import stats

# We already have the model fitted and the predictions can be made using model.predict(X)
predictions = model.predict(X)

# Calculate R^2 score
r2 = r2_score(y, predictions)

# To get p-values we need to perform an OLS regression using statsmodels because sklearn does not provide p-values
import statsmodels.api as sm

# Add a constant to the model (it's required for statsmodels OLS)
X_with_const = sm.add_constant(X)

# Fit the OLS model
ols_model = sm.OLS(y, X_with_const).fit()

# Get the summary of the model to extract the p-values
summary = ols_model.summary()

r2, summary.tables[1]


  warnings.warn("kurtosistest only valid for n>=20 ... continuing "



(0.5789897606334666, <class 'statsmodels.iolib.table.SimpleTable'>)

In [ ]:
summary.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,471.3723,0.477,987.541,0.000,470.322,472.423
angry0,0.0334,0.021,1.580,0.142,-0.013,0.080
disgust0,1.5621,3.071,0.509,0.621,-5.196,8.321
fear0,0.0705,0.144,0.489,0.635,-0.247,0.388
happy0,0.0010,0.057,0.017,0.987,-0.124,0.126
sad0,0.0682,0.182,0.375,0.715,-0.332,0.469
surprise0,0.0390,0.277,0.141,0.890,-0.570,0.648
neutral0,-0.0012,0.018,-0.066,0.948,-0.040,0.038
